In [9]:
#from msilib.schema import Directory
import tensorflow as tf
import numpy as np
#from tensorflow.keras.layers import TextVectorization
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    r"C:\Users\luis_\OneDrive\Escritorio\PT3\BDKeras",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    r"C:\Users\luis_\OneDrive\Escritorio\PT3\BDKeras",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    r"C:\Users\luis_\OneDrive\Escritorio\PT3\pop4", batch_size=batch_size
)


print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 50000 files belonging to 4 classes.
Using 40000 files for training.
Found 50000 files belonging to 4 classes.
Using 10000 files for validation.
Found 6053 files belonging to 2 classes.
Number of batches in raw_train_ds: 1250
Number of batches in raw_val_ds: 313
Number of batches in raw_test_ds: 190


In [6]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

print(raw_train_ds.map(lambda x, y: x))
print(raw_train_ds)

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label



<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>
<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>


In [ ]:
#print(raw_train_ds, "traind_ds \n")
#print(raw_val_ds, "val_ds \n")
#print(raw_test_ds, "test_ds \n")
# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

#print(train_ds, "traind_ds \n")
#print(val_ds, "val_ds \n")
#print(test_ds, "test_ds \n")


# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

#print(train_ds, "traind_ds \n")
#print(val_ds, "val_ds \n")
#print(test_ds, "test_ds \n")

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 500), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:


inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)


# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(4, activation="softmax", name="predictions")(x)
predictions=tf.reshape(predictions, shape=[1,-1])

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#*******************



In [ ]:
epochs = 5

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/5
1250/1250 [==============================] - 232s 185ms/step - loss: 1.9393 - accuracy: 0.0147 - val_loss: 1.9210 - val_accuracy: 0.0584
Epoch 2/5
1250/1250 [==============================] - 201s 161ms/step - loss: 1.9392 - accuracy: 0.0959 - val_loss: 1.9210 - val_accuracy: 0.1928
Epoch 3/5
1250/1250 [==============================] - 200s 160ms/step - loss: 1.9392 - accuracy: 0.1584 - val_loss: 1.9210 - val_accuracy: 0.2100
Epoch 4/5
1250/1250 [==============================] - 199s 159ms/step - loss: 1.9392 - accuracy: 0.2026 - val_loss: 1.9210 - val_accuracy: 0.2308
Epoch 5/5
1250/1250 [==============================] - 199s 159ms/step - loss: 1.9392 - accuracy: 0.2293 - val_loss: 1.9210 - val_accuracy: 0.2241


<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


_____________
END TO END POINT
_______________

In [ ]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")


# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)


# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)
#print(text_batch.numpy()[1])
#end_to_end_model.evaluate(text_batch.numpy()[1],label_batch.numpy()[1])



190/190 [==============================] - 6s 31ms/step - loss: 0.7748 - accuracy: 0.1788


[0.7748251557350159, 0.17875432968139648]

In [ ]:

#one_hot_word = [tf.keras.preprocessing.text.one_hot("perfect", 50)]
#pad_word = tf.keras.preprocessing.sequence.pad_sequences(one_hot_word)
#result = model.predict(pad_word)

"""
for text_batch, label_batch in test_ex.take(1):
    for i in range(0,7):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])
"""        

#len(end_to_end_model.predict(test_ex))
#end_to_end_model.predict(test_ex)
#*****

#****
#probabilities = end_to_end_model.predict(
#    [["Its the better movie that i have seen"]]
#)
probabilities = end_to_end_model.predict([input("Its the better movie that i have seen")])
print(probabilities)

result = probabilities
print("Resultado 1", result[0][0])
print("Resultado 2", result[0][1])
print("Resultado 3", result[0][2])
print("Resultado 4", result[0][3])




1/1 [==============================] - 0s 185ms/step
[[0.24999955 0.24999893 0.2500007  0.2500008 ]]
Resultado 1 0.24999955
Resultado 2 0.24999893
Resultado 3 0.2500007
Resultado 4 0.2500008


_________________________
CONVERTIR MODELO A TENSORFLOW
________________________________

In [ ]:
# Create a model using high-level tf.keras.* APIs
#from tflite_model_maker import ExportFormat
#model.save(r'C:\Users\luis_\OneDrive\Escritorio\PT3\Modelos TensorFlow Elite\model99.tflite')

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1]),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)
])
model.compile(optimizer='sgd', loss='mean_squared_error') # compile the model= sgd
model.fit(x=[-1, 0, 1], y=[-3, -1, 1], epochs=5) # train the model
# (to generate a SavedModel) tf.saved_model.save(model, 'model5.tflite')

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open(r'C:\Users\luis_\OneDrive\Escritorio\PT3\Modelos TensorFlow Elite\model.tflite', 'wb') as f:
  f.write(tflite_model)

model.save(r'C:\Users\luis_\OneDrive\Escritorio\PT3\Modelos TensorFlow Elite\model23.tflite')

#tf.saved_model.save(model, r'C:\Users\luis_\OneDrive\Escritorio\PT3\Modelos TensorFlow Elite\model5.tflite')

Epoch 1/5
1/1 [==============================] - 1s 689ms/step - loss: 5.9335
Epoch 2/5
1/1 [==============================] - 0s 11ms/step - loss: 5.4239
Epoch 3/5
1/1 [==============================] - 0s 10ms/step - loss: 5.0060
Epoch 4/5
1/1 [==============================] - 0s 10ms/step - loss: 4.6548
Epoch 5/5
1/1 [==============================] - 0s 37ms/step - loss: 4.3547
INFO:tensorflow:Assets written to: C:\Users\luis_\AppData\Local\Temp\tmp_3juyj1s\assets


INFO:tensorflow:Assets written to: C:\Users\luis_\AppData\Local\Temp\tmp_3juyj1s\assets


INFO:tensorflow:Assets written to: C:\Users\luis_\OneDrive\Escritorio\PT3\Modelos TensorFlow Elite\model23.tflite\assets


INFO:tensorflow:Assets written to: C:\Users\luis_\OneDrive\Escritorio\PT3\Modelos TensorFlow Elite\model23.tflite\assets


In [ ]:
new_model = keras.models.load_model(r'C:\Users\luis_\OneDrive\Escritorio\PT3\Modelos TensorFlow Elite\model23.tflite')
# Convert the model.
converter2 = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model2 = converter2.convert()

# Save the model.
with open(r'C:\Users\luis_\OneDrive\Escritorio\PT3\Modelos TensorFlow Elite\model26.tflite', 'wb') as f:
  f.write(tflite_model2)

INFO:tensorflow:Assets written to: C:\Users\luis_\AppData\Local\Temp\tmpxkpvsd0v\assets


INFO:tensorflow:Assets written to: C:\Users\luis_\AppData\Local\Temp\tmpxkpvsd0v\assets


In [ ]:
#model.export(export_dir='mobilebert/')


AttributeError: 'Sequential' object has no attribute 'export'